# Hyperparameter tuning

In [1]:
from google.colab import drive
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Load data from the iris dataset. It is already split into training and test sets and scaled.

In [3]:
X_train = pd.read_pickle('/content/drive/My Drive/IAAE/data/iris_Xtrain_sc.pkl')
y_train = pd.read_pickle('/content/drive/My Drive/IAAE/data/iris_ytrain.pkl')
X_test = pd.read_pickle('/content/drive/My Drive/IAAE/data/iris_Xtest_sc.pkl')
y_test = pd.read_pickle('/content/drive/My Drive/IAAE/data/iris_ytest.pkl')

## Tuning hyperparameters using the training set (not reliable!)

We will build different models on the training set and will estimate their performance on the same training set.

In [4]:
for k in [3,5,7,9,11,13,15]:
  knn_mod = KNeighborsClassifier(n_neighbors=k)
  knn_mod.fit(X_train,y_train)
  y_pred = knn_mod.predict(X_train)
  acc = accuracy_score(y_train,y_pred).round(2)
  print('Accuracy with',k,'neightbors:',acc)

Accuracy with 3 neightbors: 0.98
Accuracy with 5 neightbors: 0.98
Accuracy with 7 neightbors: 0.98
Accuracy with 9 neightbors: 0.99
Accuracy with 11 neightbors: 0.99
Accuracy with 13 neightbors: 0.99
Accuracy with 15 neightbors: 0.98


We obtain the best result with k=9, k= 11 and k=13. We choose k=13 because it is the simpler model.

In [5]:
best_model = KNeighborsClassifier(n_neighbors=13)
best_model.fit(X_train,y_train)

KNeighborsClassifier(n_neighbors=13)

Use the test set to get the final estimate of the performance of the model on new data.

In [6]:
  y_pred = best_model.predict(X_test)
  acc = accuracy_score(y_test,y_pred).round(2)
  print('Accuracy:',acc)

Accuracy: 0.91



This result shows the risk we take by using the same data for training and tuning hyperparameters. Our estimates of performance are very high for all the values of k and they are not good indicators for the generalization power of the model. When tested with new data, performance drops significantly.


## Tuning hyperparameters automatically

Define a dictionary with the names and values of the hyperparameters to include in the grid.

In [7]:
param_grid = {'n_neighbors':[3,5,7,9,11,13,15]}

Create a ``GridSearcCV`` object to automatically tune the hyperparameters.

We indicate the classifier, the grid, the evalution metric and the level of verbosity.

Depending on the amount of data and the number of hyperparameters this can be a very computer-intensive method so that we also specify that all the available processors should be used setting ``n_jobs`` to -1.

In [8]:
grid = GridSearchCV(KNeighborsClassifier(), param_grid, scoring='accuracy', verbose=3, n_jobs=-1)

Fit the grid, i.e., search among all the combinations of hyperparameters in the grid and print some results.

In [9]:
grid.fit(X_train,y_train)
print('Best hyperparameters: ',grid.best_params_)
print('Best mean CV score:', grid.best_score_.round(2))

Fitting 5 folds for each of 7 candidates, totalling 35 fits
Best hyperparameters:  {'n_neighbors': 7}
Best mean CV score: 0.96


Let's convert the results to a dataframe to take a look to the full set of results.

This can be useful if we decide not to select the absolute best. For example, because it is very close to the second or third best and these correspond to simpler models.

In [10]:
res = pd.DataFrame(grid.cv_results_)

In [11]:
res

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.006861,0.006103,0.007939,0.004611,3,{'n_neighbors': 3},0.952381,1.000000,0.904762,1.000000,0.904762,0.952381,0.042592,4
1,0.010523,0.010779,0.014217,0.008267,5,{'n_neighbors': 5},1.000000,0.952381,0.904762,1.000000,0.904762,0.952381,0.042592,4
2,0.004303,0.000796,0.006702,0.002848,7,{'n_neighbors': 7},1.000000,1.000000,0.904762,1.000000,0.904762,0.961905,0.046657,1
3,0.005546,0.004036,0.013048,0.002788,9,{'n_neighbors': 9},0.952381,1.000000,0.952381,1.000000,0.904762,0.961905,0.035635,1
4,0.003456,0.000153,0.009875,0.002451,11,{'n_neighbors': 11},0.952381,1.000000,0.904762,1.000000,0.952381,0.961905,0.035635,1
5,0.009579,0.005317,0.007881,0.003304,13,{'n_neighbors': 13},1.000000,1.000000,0.809524,1.000000,0.952381,0.952381,0.073771,4
6,0.006960,0.004496,0.007581,0.003900,15,{'n_neighbors': 15},0.904762,1.000000,0.809524,0.952381,0.952381,0.923810,0.064594,7


We fit the model to the full training data using the best value for the hyperparameter and then we apply the model to the test data to get the final performance score.

In [12]:
best_model = KNeighborsClassifier(n_neighbors=7)
best_model.fit(X_train,y_train)

KNeighborsClassifier(n_neighbors=7)

In [13]:
y_pred = best_model.predict(X_test)
acc = accuracy_score(y_test,y_pred).round(2)
print('Accuracy:',acc)

Accuracy: 0.96
